In [1]:
import os

import pandas as pd
import autonormalize as an

In [5]:
pollution_df = pd.read_csv(os.path.join(os.getcwd(), 'autonormalize/downloads/pollution.csv'))
pollution_df = pollution_df.drop(columns=pollution_df.columns[9:])
pollution_df = pollution_df.drop(range(1000000, 1746661))
pollution_df = pollution_df.drop(columns=['Date Local'])
pollution_df = pollution_df.drop_duplicates()

In [7]:
deps_exact = an.find_dependencies(pollution_df, 1.00)
deps_approx = an.find_dependencies(pollution_df, 0.96)
print("Exact dependencies...")
print(deps_exact)
print("\nApproximate dependencies...")
print(deps_approx)

100%|██████████| 8/8 [00:23<00:00,  2.88s/it]

Exact dependencies...
 --> Unnamed: 0
 {County,County Code}  {County,Site Num}  {Address}  {City,County}  {State}  --> State Code
 {County,Site Num}  {Address}  {City,County}  {County,State}  {County,State Code}  --> County Code
 {Address}  --> Site Num
 {State,Site Num,County Code}  {County,Site Num}  {State Code,Site Num,County Code}  --> Address
 {County,County Code}  {County,Site Num}  {Address}  {State Code}  {City,County}  --> State
 {Address}  {State,County Code}  {State Code,County Code}  --> County
 {State,Site Num,County Code}  {Address}  {County,Site Num}  {State Code,Site Num,County Code}  --> City

Approximate dependencies...
 --> Unnamed: 0
 {City,Site Num}  {Address}  {State}  {City,County Code}  {Site Num,County Code}  {County}  --> State Code
 {City,Site Num}  {Address}  {City,State}  {City,State Code}  {County}  --> County Code
 {Address}  --> Site Num
 {City,Site Num}  {County,Site Num}  {Site Num,County Code}  --> Address
 {City,Site Num}  {Address}  {State Code}  {

In [8]:
groupings = an.normalize_dependencies(deps_approx)
for grp in groupings:
    print('\n~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~\n')
    print(grp)


~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Unnamed: 0
 --> Address

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> Address
 {Address}  --> Site Num
 {Address}  --> City

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> City
 --> Site Num
 {City,Site Num}  --> County

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> County
 {County}  --> County Code
 {County}  --> State

~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

 --> State
 {State}  --> State Code
